In [1]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e27e5eeb-c63a-900d-fa98-1b61822349bb)
GPU 1: Tesla T4 (UUID: GPU-6ecafca8-80d5-dea5-e86e-8b0c8e7c03b1)


In [2]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ad7m21tm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ad7m21tm
  Resolved https://github.com/huggingface/transformers.git to commit 57943630e24651e6d954b912e7fcdb2b4c719cc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7562988 sha256=b0b68994bb01c118d9f149db4504117afbe46acef53e0c4f262e15fbe09ce146
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9qg7ms5/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
  Cloning https://github.com

In [3]:
!pip install transformers --upgrade

In [4]:
!pip install -q datasets bitsandbytes einops wandb

In [5]:
!pip install -U trl transformers accelerate git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-wyd9pp9n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-wyd9pp9n
  Resolved https://github.com/huggingface/peft.git to commit 3d9ceb5162356ccb66462e13bc4937350a314a5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 3.4 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.6.0.dev0-py3-none-any.whl size=82370 sha256=d477fbd9f403a283906fdb5cd2e5fc18039e5e934ab644557c7ecd4392066526
  Stored in directory: /tmp/pip-ephem-wheel-cache-cjdbbet1/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft


In [6]:
!pip install bitsandbytes

In [7]:
from huggingface_hub import login
login()

In [8]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from transformers import TrainingArguments
from transformers import Trainer
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [9]:
df_train = pd.read_csv('/kaggle/input/medical-chatbot-dataset/train_data_chatbot.csv')
df_train.head(5)

,short_question,short_answer,tags,label
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,['rash' 'antibiotic'],1.0
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,['hepatitis b'],1.0
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,['celiac disease'],1.0
3,can i transmit genital warts seventeen years a...,famotidine pepcid products is in a drug class ...,['wart'],-1.0
4,is all vitamin d the same,hi this means you do not have hepatitis b and ...,['vitamin d'],-1.0


In [10]:
df_test = pd.read_csv('/kaggle/input/medical-chatbot-dataset/validation_data_chatbot.csv')
df_test.head(5)

,short_question,short_answer,tags,label
0,do i have a yeast infection,hi this can be a vaginal fungal infection whic...,['yeast infection'],1.0
1,does your upper backlung area hurt badly with ...,if you have a true allergy to sulfa medication...,['asthma' 'ibuprofen' 'lung' 'cough' 'inhaler'],-1.0
2,taken off paxil cold turk now on celexa having...,if overdose is suspected contact a poison cont...,['brain' 'lightheadedness' 'dizziness' 'coldne...,-1.0
3,i need to buy health insurance asap what do i ...,go to healthcare gov call my husband 407 222 9...,[''],1.0
4,what happens if a pregnant woman has genital h...,sorry for her pain i understand your feelings ...,['genital herpes' 'pregnancy'],-1.0


In [11]:
# Delete two columns
columns_to_delete = ['tags', 'label']
df_train = df_train.drop(columns=columns_to_delete)
df_test = df_test.drop(columns=columns_to_delete)

In [12]:
df_train.head(2)

,short_question,short_answer
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...


In [13]:
df_train = df_train.iloc[20000:]
df_test = df_test.iloc[5000:]

In [14]:
df_train.shape, df_test.shape

((27603, 2), (6901, 2))

In [15]:
df_train.to_csv('/kaggle/working/train-data-small.csv', index= False)
df_test.to_csv('/kaggle/working//test-data-small.csv', index= False)

In [16]:
# import os

# # Specify the path of the file you want to delete
# file_path = '/kaggle/working/train-data.csv'  # Replace with the actual path

# # Check if the file exists before attempting to delete
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File '{file_path}' has been deleted.")
# else:
#     print(f"File '{file_path}' does not exist.")


In [17]:
from datasets import load_dataset
datasets = load_dataset('csv', data_files={"train": '/kaggle/working/train-data-small.csv', "validation": '/kaggle/working/test-data-small.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d105950a4930132e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
datasets["train"][0]

{'short_question': 'i found a weird looking mole on my breast could it be skin cancer',
 'short_answer': 'definitely not a skin cancer you can be sure thank you'}

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


In [20]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [22]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [23]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["short_question"]]
    inputs = tokenizer(
        questions,
        examples["short_answer"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    return inputs

In [24]:
tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [25]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [26]:
# from evaluate import evaluator
# import evaluate
from datasets import load_metric

def compute_metrics(pred):
    squad_labels = pred.label_ids
    squad_preds = pred.predictions.argmax(-1)

    # Calculate Exact Match (EM)
    em = sum([1 if p == l else 0 for p, l in zip(squad_preds, squad_labels)]) / len(squad_labels)

    # Calculate F1-score
    f1 = f1_score(squad_labels, squad_preds, average='macro')

    return {
        'exact_match': em,
        'f1': f1
    }

In [27]:
transformers.__version__

'4.33.0.dev0'

In [28]:
from transformers import TrainingArguments

output_dir = "/kaggle/working/"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
evaluation_strategy="epoch"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
weight_decay=0.01
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"
num_train_epochs = 5
#clf_metrics = evaluate.combine(["f1", "precision", "recall"])

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    evaluation_strategy=evaluation_strategy,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    num_train_epochs=num_train_epochs,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    logging_strategy="epoch", #Extra: to log training data stats for loss
    #compute_metrics=compute_metrics,
)

In [29]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example["short_question"])):
        text = f"### Question: {example['short_question'][i]}\n ### Answer: {example['short_answer'][i]}"
        output_texts.append(text)
    
    return output_texts

In [30]:
from trl import SFTTrainer

max_seq_length = 512
#collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    formatting_func=formatting_prompts_func,
    #data_collator=data_collator,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    peft_config=peft_config,
)



/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [31]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [32]:
trainer.train() 

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,2.236400,2.169756


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=100, training_loss=2.236356506347656, metrics={'train_runtime': 19118.7676, 'train_samples_per_second': 0.167, 'train_steps_per_second': 0.005, 'total_flos': 1.075603499286528e+16, 'train_loss': 2.236356506347656, 'epoch': 0.12})

In [33]:
# from trl import SFTTrainer

# max_seq_length = 512

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets["validation"],
#     peft_config=peft_config,
#     #dataset_text_field="text",
#     formatting_func=formatting_prompts_func,
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     data_collator=data_collator,
# )

In [34]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 8.76


In [40]:
eval_results

{'eval_loss': 2.1697559356689453,
 'eval_runtime': 10948.6822,
 'eval_samples_per_second': 0.63,
 'eval_steps_per_second': 0.039,
 'epoch': 0.12}

In [35]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [36]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [62]:
generation_config = model.generation_config
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 32
generation_config.use_cache = False
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [63]:
def generate_prompt(prompt: str) -> str:
    return f"""
    <human>: {prompt}
    <assistant>: 
    """.strip()


In [64]:
model.device

device(type='cuda', index=0)

In [58]:
prompt = "un antibiotique par voie intraveineuse peut-il vous donner une éruption cutanée quelques jours plus tard?"

prompt = generate_prompt(prompt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )

response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

<human>: un antibiotique par voie intraveineuse peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <assistant>: Oui, il peut.
    <human>: un antibiotique par voie orale peut-il vous donner une éruption cutanée quelques jours plus tard?
    <ass


In [65]:
prompt = "quelles sont les restrictions alimentaires pour le gluten de la maladie coeliaque?"

prompt = generate_prompt(prompt)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
    )

response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print(response)

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 14.76 GiB total capacity; 13.20 GiB already allocated; 9.75 MiB free; 13.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from huggingface_hub import login
login()

In [66]:
model.push_to_hub("llama2-qlora-finetunined-ChatMed-french")

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ndams/llama2-qlora-finetunined-ChatMed-french/commit/1a8ef01b145896b7cf0edbc2affcd0135cabe2ff', commit_message='Upload model', commit_description='', oid='1a8ef01b145896b7cf0edbc2affcd0135cabe2ff', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# import torch
# from peft import PeftConfig, PeftModel
# from transformers import GenerationConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# MODEL_NAME = "Sandiago21/falcon-7b-prompt-answering"
# BASE_MODEL = "tiiuae/falcon-7b"

# compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=True,
# )

# model = AutoModelForCausalLM.from_pretrained(
#     BASE_MODEL,
#     quantization_config=bnb_config,
#     device_map="auto",
#     trust_remote_code=True,
# )

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# model = PeftModel.from_pretrained(model, MODEL_NAME)

# generation_config = model.generation_config
# generation_config.top_p = 0.7
# generation_config.num_return_sequences = 1
# generation_config.max_new_tokens = 32
# generation_config.use_cache = False
# generation_config.pad_token_id = tokenizer.eos_token_id
# generation_config.eos_token_id = tokenizer.eos_token_id

# model.eval()
# if torch.__version__ >= "2":
#     model = torch.compile(model)


In [ ]:
# prompt = "What is the capital city of Greece and with which countries does Greece border?"

# prompt = generate_prompt(prompt)
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# input_ids = input_ids.to(model.device)

# with torch.no_grad():
#     outputs = model.generate(
#         input_ids=input_ids,
#         generation_config=generation_config,
#         return_dict_in_generate=True,
#         output_scores=True,
#     )

# response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
# print(response)

# >>> The capital city of Greece is Athens and it borders Albania, Bulgaria, Macedonia, and Turkey.


In [ ]:
class CFG:
    # LLMs
    model_name = 'llama2-13b' # wizardlm, bloom, falcon, llama2-7b, llama2-13b
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15    

    # splitting
    split_chunk_size = 800
    split_overlap = 0
    
    # embeddings
    embeddings_model_repo = 'hkunlp/instructor-base'    

    # similar passages
    k = 3
    
    # paths
    PDFs_path = '/kaggle/input/medbooks/'
    
    Embeddings_path =  '/kaggle/working/medbooks-vectordb-chroma'
    Persist_directory = './medbooks-vectordb-chroma'  

In [ ]:
def get_model(model = CFG.model_name):

    print('\nDownloading model: ', model, '\n\n')

    if model == 'llama2-7b':
        model_repo = 'daryl149/llama-2-7b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
            )
#         bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.float16,
#             )

#         model = AutoModelForCausalLM.from_pretrained(
#             model_name,
#             quantization_config=bnb_config,
#             trust_remote_code=True
#         )
#         model.config.use_cache = False
        
        max_len = 512

    elif model == 'llama2-13b':
        model_repo = 'daryl149/llama-2-13b-chat-hf'
        
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            load_in_4bit=True,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
            )
        
        max_len = 512
        
    else:
        print("Not implemented model (tokenizer and backbone)")

    return tokenizer, model, max_len

In [ ]:
%%time

tokenizer, model, max_len = get_model(model = CFG.model_name)

In [ ]:
datasets['train'][0]

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["short_question"]]
    inputs = tokenizer(
        questions,
        examples["short_answer"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    return inputs

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [ ]:
tokenized_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)

In [ ]:
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     trust_remote_code=True
# )
# model.config.use_cache = False

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
class Args:
    
    output_dir = "/kaggle/working/"
    per_device_train_batch_size = 4
    gradient_accumulation_steps = 4
    optim = "paged_adamw_32bit"
    save_steps = 100
    logging_steps = 10
    learning_rate = 2e-4
    max_grad_norm = 0.3
    max_steps = 100
    warmup_ratio = 0.03
    lr_scheduler_type = "constant"

In [ ]:
def create_training_arguments(Args):
    return TrainingArguments(
        output_dir=Args.output_dir,
        per_device_train_batch_size=Args.per_device_train_batch_size,
        gradient_accumulation_steps=Args.gradient_accumulation_steps,
        optim=Args.optim,
        save_steps = Args.save_steps,
        logging_steps= Args.logging_steps,
        learning_rate=Args.learning_rate,
        max_grad_norm=Args.max_grad_norm,
        max_steps=Args.max_steps,
        warmup_ratio=Args.warmup_ratio,
        lr_scheduler_type=Args.lr_scheduler_type,
    )

In [ ]:
training_arguments = create_training_arguments(Args)

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
#from datasets import save_dataset
tokenized_datasets.save_to_disk("/kaggle/working/tokenized_datasets.hf")

In [ ]:
tokenized = load_dataset("json", data_files="/kaggle/working/tokenized_datasets.hf")

In [ ]:
len(tokenized_datasets['train'])

In [ ]:
trainer = Trainer(
    model=model,
    args=create_training_arguments(training_arguments),
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

trainer.train()